# Feature detection and matching

## Motivation

Features are used for:
- Automatic panoramas
- Indexing and content-based retrieval (ex: Google images,...)
- Object recognition
- 3D reconstruction
- Motion tracking
- Robot navigation
- ...

## Which features are good to choose

*Choose features that are invariant to transformations*

**Invariant local features**
- Corners
- Blobs
- ...

**Advantages of local features**:
- Locality: robust to occlusion and clutter
- Quantity: hundreds or thousands in a single image
- Distinctiveness: can differentiate a large database of objects
- Efficiency: real-time performance achievale

*What about edges*
- Edges can be invariant to brightness changes but typically not invariant to other transformations

*Note on transformations*:
- Geometric invariance: translation, rotation, scale
- Photometric invariance: brightness, exposure

**Idea to measure the uniqueness of a feature**
- Consider a small window of pixels
- "Observe" how does the window change when you shift it
    - "flat" region: no change in all directions
    - "edge" region: no change along the edge direction
    - "corner": significant change in all directions

# Harris corner detection

Consider shifting the window $W$ by $(u,v)$

Sum of squared differences **(SSD "error")**:
$$
E(u,v) = \sum_{(x,y)\in W} (I(x+u, y+v) - I(x, y))^2
$$

If the motion $(u,v)$ is small, we can calculate the **approximation of** $I(x+u,y+v)$ using Taylor Series:
$$
\begin{aligned}
    I(x+u,y+v)
        & \approx I(x,y) + \frac{\partial I}{\partial x}u + \frac{\partial I}{\partial y}v \\
        & \approx I(x,y) + I_x u + I_y v   \\
        & \approx I(x,y) + \begin{bmatrix}I_x & I_y\end{bmatrix}\begin{bmatrix}u \\ v\end{bmatrix}
\end{aligned}
$$

Plug the approximate $I(x+u,y+v)$ formula into $E$, we have:
$$
\begin{aligned}
    E(u,v)
        & = \sum_{(x,y)\in W} (I(x+u, y+v) - I(x, y))^2 \\
        & \approx \sum_{(x,y) \in W} (I(x,y) + I_x u + I_y v - I(x,y))^2 \\
        & \approx \sum_{(x,y) \in W} (I_x u + I_y v)^2 \\
        & \approx \sum_{(x,y) \in W} (I_x^2 u^2 + 2I_xI_yuv + I_y^2 v^2)
\end{aligned}
$$

Set:
$$
A = \sum_{(x,y) \in W} I_x^2
$$
$$
B = \sum_{(x,y) \in W} I_xI_y
$$
$$
C = \sum_{(x,y) \in W} I_y^2
$$

We have:
$$
\begin{aligned}
    E(x,y) 
    & \approx A u^2 + 2Buv + C v^2 \\
    & \approx \begin{bmatrix}u & v\end{bmatrix}
      \begin{bmatrix}A & B \\ B & C\end{bmatrix}
      \begin{bmatrix}u & v\end{bmatrix} 
\end{aligned}
$$

https://courses.cs.washington.edu/courses/cse455/09wi/Lects/lect6.pdf